# Inference

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

In [1]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for NER with your model, and pass your text to it:

In [2]:
text = "My name is Wolfgang and I live in Berlin"
from transformers import pipeline

classifier = pipeline("ner", model="ner_conll2003_model/checkpoint-1756")
result = classifier(text)
result = result[::-1]

/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [3]:
result

[{'entity': 'B-LOC',
  'score': 0.9958467,
  'index': 9,
  'word': 'berlin',
  'start': 34,
  'end': 40},
 {'entity': 'B-PER',
  'score': 0.9844818,
  'index': 4,
  'word': 'wolfgang',
  'start': 11,
  'end': 19}]

In [4]:
# Function to wrap entities with markup
def wrap_entities(text, entities):
    marked_text = text
    for entity in entities:
        start = entity['start']
        end = entity['end']
        entity_type = entity['entity']
        marked_text = marked_text[:start] + f"<{entity_type}>{text[start:end]}</{entity_type}>" + marked_text[end:]
    return marked_text

marked_text = wrap_entities(text, result)
print(marked_text)

My name is <B-PER>Wolfgang</B-PER> and I live in <B-LOC>Berlin</B-LOC>


You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return PyTorch tensors:

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ner_conll2003_model/checkpoint-1756")
inputs = tokenizer(text, return_tensors="pt")

Pass your inputs to the model and return the `logits`:

In [6]:
import torch
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("ner_conll2003_model/checkpoint-1756")
with torch.no_grad():
    logits = model(**inputs).logits

Get the class with the highest probability, and use the model's `id2label` mapping to convert it to a text label:

In [7]:
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class

['O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'B-LOC', 'I-MISC']

In [8]:
len(inputs['input_ids'][0])

11

In [9]:
len(predicted_token_class)

11

# Use bert-base-NER

In [10]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [11]:
ner_results = ner_results[::-1]
marked_text = wrap_entities(example, ner_results)
print(marked_text)

My name is <B-PER>Wolfgang</B-PER> and I live in <B-LOC>Berlin</B-LOC>
